# Проект кинопроката
Таблица `mkrf_movies` содержит информацию из реестра прокатных удостоверений. У одного фильма может быть несколько прокатных удостоверений. 

- `title` — название фильма;
- `puNumber` — номер прокатного удостоверения;
- `show_start_date` — дата премьеры фильма;
- `type` — тип фильма;
- `film_studio` — студия-производитель;
- `production_country` — страна-производитель;
- `director` — режиссёр;
- `producer` — продюсер;
- `age_restriction` — возрастная категория;
- `refundable_support` *—* объём возвратных средств государственной поддержки;
- `nonrefundable_support` *—* объём невозвратных средств государственной поддержки;
- `financing_source` *—* источник государственного финансирования;
- `budget` *—* общий бюджет фильма;
- `ratings` *—* рейтинг фильма на КиноПоиске;
- `genres` *—* жанр фильма.

Обратите внимание, что столбец `budget` **уже включает в себя полный объём государственной поддержки. Данные в этом столбце указаны только для тех фильмов, которые получили государственную поддержку. 

Таблица `mkrf_shows` содержит сведения о показах фильмов в российских кинотеатрах.

- `puNumber` — номер прокатного удостоверения;
- `box_office` — сборы в рублях.

In [80]:
import os
import pandas as pd
import numpy as np
import re
import datetime
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

## Базовые задачи

### Опишите проблемы типов имеющихся данных и варианты исправления

In [81]:
mkrf_movies = pd.read_csv('mkrf_movies.csv', dtype=str)
mkrf_movies.describe(include='all').T

,count,unique,top,freq
title,7486,6772,День святого Валентина_,5
puNumber,7486,7484,221054410,2
show_start_date,7486,1965,2010-12-17T12:00:00.000Z,96
type,7486,8,Художественный,4520
film_studio,7468,5490,"Киностудия ""Мосфильм""",341
production_country,7484,950,США,2172
director,7477,4811,О.Семёнова,47
producer,6918,5349,П.Смирнов,67
age_restriction,7486,5,«16+» - для детей старше 16 лет,2851
refundable_support,332,34,0.0,221


In [82]:
mkrf_shows = pd.read_csv('mkrf_shows.csv', dtype=str)
mkrf_shows.describe(include='all').T

,count,unique,top,freq
puNumber,3158,3158,111000113,1
box_office,3158,2938,0.0,24


In [83]:
mkrf_merge = mkrf_movies.merge(mkrf_shows, how='left', on='puNumber').copy()
mkrf_merge.shape

(7486, 16)

In [84]:
mkrf_merge.head(2).T

,0,1
title,Открытый простор,Особо важное задание
puNumber,221048915,111013716
show_start_date,2015-11-27T12:00:00.000Z,2016-09-13T12:00:00.000Z
type,Художественный,Художественный
film_studio,"Тачстоун Пикчерз, Кобальт Пикчерз, Бикон Пикче...","Киностудия ""Мосфильм"""
production_country,США,СССР
director,Кевин Костнер,Е.Матвеев
producer,"Дэвид Валдес, Кевин Костнер, Джейк Эбертс",NaN
age_restriction,«18+» - запрещено для детей,«6+» - для детей старше 6 лет
refundable_support,NaN,NaN


In [85]:
mkrf_merge_backup = mkrf_merge.copy()
mkrf_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7486 entries, 0 to 7485
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   title                  7486 non-null   object
 1   puNumber               7486 non-null   object
 2   show_start_date        7486 non-null   object
 3   type                   7486 non-null   object
 4   film_studio            7468 non-null   object
 5   production_country     7484 non-null   object
 6   director               7477 non-null   object
 7   producer               6918 non-null   object
 8   age_restriction        7486 non-null   object
 9   refundable_support     332 non-null    object
 10  nonrefundable_support  332 non-null    object
 11  budget                 332 non-null    object
 12  financing_source       332 non-null    object
 13  ratings                6519 non-null   object
 14  genres                 6510 non-null   object
 15  box_office           

Видим много пропусков данных, но радует, что датафремы качественно сопоставились и все 3158 значений по кассовым сборам нашли свою пару.

### Болле подробно расммотрим все признаки:

1) `title` - кроме самого названия может содержать описание, год выпуска и альтернативное название. Решения:
* избавимся от описания, заключенного в символы `\` - отправим эти данные в отдельные признаки `first_lv_desc`, `second_lv_desc`.
* избавимся от прочей информации в скобках - поместим ее в `alter_title`

Обратим внимание, что год выхода фильма указанный в названии может отличаться от данных в `show_start_date` - стоит уточнить правила работы с признаками у заказчика.

In [86]:
mkrf_merge_get_decription = mkrf_merge['title'].str.rsplit("/",expand = True)
mkrf_merge_get_decription.columns = ['title_true', 'first_lv_desc', 'second_lv_desc']
mkrf_merge = pd.concat([mkrf_merge, mkrf_merge_get_decription], axis=1)

mkrf_merge['alter_title'] = mkrf_merge['title'].str.extract(r'(\(\w+\)$)')
mkrf_merge['alter_title'] = mkrf_merge['alter_title'].str.replace(r'\W','')
mkrf_merge['title_true'] = mkrf_merge['title_true'].str.replace(r'\([\w\W]+$','')

<ipython-input-86-03c2e6b20749>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  mkrf_merge['alter_title'] = mkrf_merge['alter_title'].str.replace(r'\W','')
<ipython-input-86-03c2e6b20749>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  mkrf_merge['title_true'] = mkrf_merge['title_true'].str.replace(r'\([\w\W]+$','')


Проверим результат, посмотрев на самые короткие названия - видим, что данное решение не всегда корректно:

In [87]:
# 7478
# 3705
mkrf_merge['title_true_len'] = mkrf_merge['title_true'].str.len()
mkrf_merge.sort_values('title_true_len')[['title', 'title_true', 'first_lv_desc',
 'second_lv_desc',
 'alter_title',
 'title_true_len',
 'type',
 'film_studio',
 'production_country']].head(10)

,title,title_true,first_lv_desc,second_lv_desc,alter_title,title_true_len,type,film_studio,production_country
977,(Не)жданный принц,,None,None,NaN,0,Художественный,"Диджитал Фэктори, Европа Корп., ТФ 1 Фильм Про...",Франция
7478,(Не)идеальный мужчина,,None,None,NaN,0,Художественный,"ООО ""Нон-Стоп Продакшн""",Россия
5792,9/11,9,11,None,NaN,1,Художественный,"Блэк Беар Студиос, Филм Хауз, Спрокфеллер Пикч...",США
2241,З/Л/О: Новый вирус,З,Л,О: Новый вирус,NaN,1,Художественный,"8383 Продакшнз, Блуди Дисгастинг, Коллектив",США
1668,З/Л/О 2,З,Л,О 2,NaN,1,Художественный,"Блуди Дисгастинг, Коллектив, 8383 Продакшнз, С...",США - Канада - Индонезия
2242,З/Л/О: Новый вирус,З,Л,О: Новый вирус,NaN,1,Художественный,"8383 Продакшнз, Блуди Дисгастинг, Коллектив",США
3705,Не/смотря ни на что,Не,смотря ни на что,None,NaN,2,Художественный,"Сэвэн Пикчерс, СтудиоКанал, Зиглер Синема",Германия
3106,g (джи),g,None,None,джи,2,Художественный,"Life is Short, Pin-Up Production, DPG, Отдел 13",Россия
1711,ЖЖ,ЖЖ,None,None,NaN,2,Художественный,"ООО ""ЛВ ФИЛЬМ""",Россия
6825,Мы,Мы,None,None,NaN,2,Художественный,"Манкипо Продакшнз, Блюмхауз Продакшнз, Юниверс...",США


2) Очевидно что `puNumber` — номер прокатного удостоверения мы должны использовать как индексы. Изучим природу 2-х пар дублей и примем решение чуть позже.

In [88]:
mkrf_merge[mkrf_merge.duplicated(subset='puNumber', keep=False)]

,title,puNumber,show_start_date,type,film_studio,production_country,director,producer,age_restriction,refundable_support,nonrefundable_support,budget,financing_source,ratings,genres,box_office,title_true,first_lv_desc,second_lv_desc,alter_title,title_true_len
4638,Как жениться и остаться холостым,221154310,2010-12-17T12:00:00.000Z,Художественный,"Ше Вам, Скрипт Ассосье, Тэ Фэ 1 Фильм Продюксь...",Франция,Эрик Лартиго,"Амандин Било, Алан Шаба",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.0,"мелодрама,комедия",NaN,Как жениться и остаться холостым,None,None,NaN,32
4639,Иоанна - женщина на папском престоле /По роман...,221154310,2010-12-17T12:00:00.000Z,Художественный,"Константин Фильм, А Эр Ди Дегето Фильм, Дюне ...",Германия - Великобритания - Италия - Испания,Зенке Вортманн,"Оливер Бербен, Дорис Д.Хайнце, Фарук Элтан",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.6,"драма,мелодрама,история",NaN,Иоанна - женщина на папском престоле,По роману Донны Вулфольк Кросс,,NaN,37
5067,Анализируй это!,221054410,2010-05-25T12:00:00.000Z,Художественный,"Уорнер Бразерс, Вилладж Роудшоу Филмз ЛТД",США-Австралия,ХЭрольд Рэмис,"Джейн Розенталь, Пола Уейнстейн",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.4,"комедия,криминал",NaN,Анализируй это!,None,None,NaN,15
5068,Анализируй то!,221054410,2010-05-25T12:00:00.000Z,Художественный,"Уорнер Бразерс, Виллидж Роадшоу Пикчерз, Эн-Пи...",США,Гарольд Реймис,"Джейн Розенталь, Паул Уэйнстейн",«12+» - для детей старше 12 лет,NaN,NaN,NaN,NaN,6.8,"комедия,криминал",NaN,Анализируй то!,None,None,NaN,14


3) `show_start_date` - переведем данные в тип данных 'дата' и отбросим время. Создадим столбws `show_start_year` и `show_start_month` для будущего анализа.

In [89]:
mkrf_merge['show_start_year'] = pd.to_datetime(mkrf_merge['show_start_date']).dt.tz_localize(None).dt.year
mkrf_merge['show_start_month'] = pd.to_datetime(mkrf_merge['show_start_date']).dt.tz_localize(None).dt. month
mkrf_merge['show_start_date'] = pd.to_datetime(mkrf_merge['show_start_date']).dt.tz_localize(None).dt.date

4) Для убедимся в сопоставимости и качестве значений для признаков `type`, `age_restriction`, `financing_source` — структурирем категории, где это необходимо. Создадим числовой признак `age_limit`, на базе строкового `age_restriction`.

In [90]:
mkrf_merge['type'] = mkrf_merge['type'].str.lstrip()
mkrf_merge['type'] = mkrf_merge['type'].str.rstrip()

In [91]:
mkrf_merge['age_limit'] = mkrf_merge['age_restriction'].str[1:3]
mkrf_merge['age_limit'] = mkrf_merge['age_limit'].str.replace('\+','')
mkrf_merge['age_limit'].value_counts()

<ipython-input-91-1b9d2f96d961>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  mkrf_merge['age_limit'] = mkrf_merge['age_limit'].str.replace('\+','')


16    2851
18    1605
12    1593
0      811
6      626
Name: age_limit, dtype: int64

5) Посмотрим на значения в `ratings`. Изучим фильмы которые оценены процентно на сайте КиноПоиска - обнаружить какую-то логику не получилось. Создадим для этих фильмов отдельный набор и позже пример решение, как работать с этими данными.

In [92]:
mkrf_merge[(~mkrf_merge['ratings'].isna()) \
           & (mkrf_merge['ratings'].str.contains('%'))].sort_values(by='ratings')[[
'title_true',
'director','producer',                                                                                                    
'title',
'type',
'ratings',
'genres',
'box_office',
'alter_title',
'show_start_year',
'show_start_month']].sample(7)

,title_true,director,producer,title,type,ratings,genres,box_office,alter_title,show_start_year,show_start_month
5455,Вечный холод,А.Мигачев,И.Бачурин,Вечный холод,Художественный,83%,"фэнтези,боевик,триллер",NaN,NaN,2017,8
3139,Я умею вязать,Н.Степанова при участии С.Иванова,"С.Кикнавелидзе, Д.Улюкаев",Я умею вязать,Художественный,90%,драма,801370.0,NaN,2015,11
6426,Памятные даты России. Партизаны и подпольщики,"Н.Иванова, А.Кузнецова",В.Коханович,Памятные даты России. Партизаны и подпольщики,Документальный,91%,"драма,биография",NaN,NaN,2018,11
259,Прошлое,Эктор Бабенко,"Эктор Бабенко, Оскар Крамер, Хуго Сидмэн",Прошлое,Художественный,99%,"ужасы,триллер,драма",NaN,NaN,2015,7
6737,Колесо времени,Б.Куломзин,Б.Куломзин,Колесо времени,Художественный,98%,"фэнтези,приключения",34992.0,NaN,2019,3
5910,Мира,Д.Шабаев,Н.Мокрицкая,Мира,Документальный,94%,драма,NaN,NaN,2018,6
5332,Добыча.,Патрик Сиверсен,"Закари Ти Брайан, Кристофер",Добыча.,Художественный,88%,"фантастика,боевик,триллер",NaN,NaN,2011,6


In [93]:
mkrf_no_rating_backup = mkrf_merge[(~mkrf_merge['ratings'].isna()) \
           & (mkrf_merge['ratings'].str.contains('%'))]

mkrf_no_rating_backup.shape

(29, 24)

6)  Далее нам надо привести следующие столбцы к числовому формату:

* 'refundable_support',
* 'nonrefundable_support',
* 'budget',
* 'ratings',
* 'box_office',
* age_limit

In [94]:
for col in ['refundable_support',
 'nonrefundable_support',
 'budget',
 'ratings',
 'box_office',
'age_limit']:
    mkrf_merge[col] = mkrf_merge[col].str.rstrip()
    mkrf_merge[col] = mkrf_merge[col].str.lstrip()
    mkrf_merge[col] = pd.to_numeric(mkrf_merge[col], errors='coerce')

In [95]:
mkrf_merge.sample(5).T

,6120,998,6057,5720,5801
title,Слендермен,Призрачный патруль /По мотивам одноименного ко...,Проигранное место,12 мелодий любви,Взрослые игры
puNumber,121017818,121012513,111012218,121001918,121006318
show_start_date,2018-08-16,2013-05-07,2018-10-25,2018-02-08,2018-04-05
type,Художественный,Художественный,Художественный,Художественный,Художественный
film_studio,"Митолоджи Интертейнмент, Мэдхаус Интертейнмент","Дак Хос Интертейнмент, Ориджинал Филм, Эр Эс И...","ООО ""ВВП Альянс""","Вестенд Фильм, Ле 13 Фильм, Дэвис-Фильм, Франс...","Диабло Интертеймент 2, Металвок Пикчерз, Рух Х..."
production_country,США,США,Россия,Франция,США
director,Силвейн Уайт,Роберт Швентке,Н.Михалкова,Клод Лелуш,Присцилла Камерон
producer,"Джеймс Вандербилт, Робин Мейсингер, Сара Сноу,...","Нил Х.Мориц, Майк Ричардсон, Майкл Фоттрел","Н.Михалкова, А.Михалкова, М.Королев, К.Комолова","Клод Лелуш, Сэмюэл Халида","Эрик Б. Флайшмен, Брэндон Джеймс"
age_restriction,«16+» - для детей старше 16 лет,«12+» - для детей старше 12 лет,«16+» - для детей старше 16 лет,«18+» - запрещено для детей,«18+» - запрещено для детей
refundable_support,NaN,NaN,0.0,NaN,NaN


7) Так же мы видим, что над фильмом могут работать большие коллективы: несколько студий, стран, режисеров и продюссеров. Весьма вероятно, что это важный фактор при оценке сложности работы над фильмом. Произведем подсчет участников для каждого фильма в признаках
* 'film_studio',
* 'production_country',
* 'director',
* 'producer'

Обратим внимание, что перечисление везде ведется через запятую, но в случае со странами, запятую иногда заменяют тире.

In [96]:
for col in ['film_studio', 'production_country', 'director', 'producer']:
    mkrf_merge[f'counts_'+col] = mkrf_merge[col].str.count(',')
    
    
mkrf_merge[f'counts_production_country'] += mkrf_merge['production_country'].str.count('-')

In [97]:
mkrf_merge.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
title,7486,6772,День святого Валентина_,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
puNumber,7486,7484,221054410,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
show_start_date,7486,1965,2010-12-17,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,7486,6,Художественный,5909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
film_studio,7468,5490,"Киностудия ""Мосфильм""",341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
production_country,7484,950,США,2172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
director,7477,4811,О.Семёнова,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN
producer,6918,5349,П.Смирнов,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age_restriction,7486,5,«16+» - для детей старше 16 лет,2851,NaN,NaN,NaN,NaN,NaN,NaN,NaN
refundable_support,332.0,NaN,NaN,NaN,11864457.831325,24916555.263644,0.0,0.0,0.0,15000000.0,180000000.0


In [98]:
mkrf_merge[mkrf_merge['counts_production_country'] == 9]['production_country'][1724]

'Мексика - Эквадор - Канада - США - Франция - Малайзия - Италия - Аргентина - Германия - Индия  '

In [99]:
mkrf_merge[mkrf_merge['counts_producer'] == 14]['producer'][1724]

"Натали Галазка, Кристиан Гемеинер, Родриго Доносо,  Милко Коронед, Аншут Кумар, Ваи Ленг, Марк Никелсбург, Мелисса С.О'Брайн, Флориан Сиегл, Стефан Сках, Джеймс Стюарт, Доменико Тискони, Чеонг, Федерико Эйер, Джереми Ясхес"

In [188]:
w_df = mkrf_merge.copy()

### Исследуйте дубликаты и пропуски. Какова их природа в этом наборе данных? Можно ли это исправить?

Смотрим на дубли по признаку `puNumber`:

Очевидный дубль `Анализируй это!` - просто проверим в [источнике](https://www.kinopoisk.ru/film/1594/)

In [189]:
w_df[w_df.duplicated(subset='puNumber', keep=False)]

,title,puNumber,show_start_date,type,film_studio,production_country,director,producer,age_restriction,refundable_support,nonrefundable_support,budget,financing_source,ratings,genres,box_office,title_true,first_lv_desc,second_lv_desc,alter_title,title_true_len,show_start_year,show_start_month,age_limit,counts_film_studio,counts_production_country,counts_director,counts_producer
4638,Как жениться и остаться холостым,221154310,2010-12-17,Художественный,"Ше Вам, Скрипт Ассосье, Тэ Фэ 1 Фильм Продюксь...",Франция,Эрик Лартиго,"Амандин Било, Алан Шаба",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.0,"мелодрама,комедия",NaN,Как жениться и остаться холостым,None,None,NaN,32,2010,12,16,4.0,0.0,0.0,1.0
4639,Иоанна - женщина на папском престоле /По роман...,221154310,2010-12-17,Художественный,"Константин Фильм, А Эр Ди Дегето Фильм, Дюне ...",Германия - Великобритания - Италия - Испания,Зенке Вортманн,"Оливер Бербен, Дорис Д.Хайнце, Фарук Элтан",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.6,"драма,мелодрама,история",NaN,Иоанна - женщина на папском престоле,По роману Донны Вулфольк Кросс,,NaN,37,2010,12,16,6.0,3.0,0.0,2.0
5067,Анализируй это!,221054410,2010-05-25,Художественный,"Уорнер Бразерс, Вилладж Роудшоу Филмз ЛТД",США-Австралия,ХЭрольд Рэмис,"Джейн Розенталь, Пола Уейнстейн",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.4,"комедия,криминал",NaN,Анализируй это!,None,None,NaN,15,2010,5,16,1.0,1.0,0.0,1.0
5068,Анализируй то!,221054410,2010-05-25,Художественный,"Уорнер Бразерс, Виллидж Роадшоу Пикчерз, Эн-Пи...",США,Гарольд Реймис,"Джейн Розенталь, Паул Уэйнстейн",«12+» - для детей старше 12 лет,NaN,NaN,NaN,NaN,6.8,"комедия,криминал",NaN,Анализируй то!,None,None,NaN,14,2010,5,12,4.0,0.0,0.0,1.0


In [190]:
# удалим строку с индексом 5068
w_df = w_df.drop(5068)
w_df[w_df.duplicated(subset='puNumber', keep=False)]

,title,puNumber,show_start_date,type,film_studio,production_country,director,producer,age_restriction,refundable_support,nonrefundable_support,budget,financing_source,ratings,genres,box_office,title_true,first_lv_desc,second_lv_desc,alter_title,title_true_len,show_start_year,show_start_month,age_limit,counts_film_studio,counts_production_country,counts_director,counts_producer
4638,Как жениться и остаться холостым,221154310,2010-12-17,Художественный,"Ше Вам, Скрипт Ассосье, Тэ Фэ 1 Фильм Продюксь...",Франция,Эрик Лартиго,"Амандин Било, Алан Шаба",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.0,"мелодрама,комедия",NaN,Как жениться и остаться холостым,None,None,NaN,32,2010,12,16,4.0,0.0,0.0,1.0
4639,Иоанна - женщина на папском престоле /По роман...,221154310,2010-12-17,Художественный,"Константин Фильм, А Эр Ди Дегето Фильм, Дюне ...",Германия - Великобритания - Италия - Испания,Зенке Вортманн,"Оливер Бербен, Дорис Д.Хайнце, Фарук Элтан",«16+» - для детей старше 16 лет,NaN,NaN,NaN,NaN,7.6,"драма,мелодрама,история",NaN,Иоанна - женщина на папском престоле,По роману Донны Вулфольк Кросс,,NaN,37,2010,12,16,6.0,3.0,0.0,2.0


Другая пара явно разные фильмы, мы не хотим пока терять данные, поэтому просто добавим дополнительный символ в один из номеров прокатного удостоверения (предварительно убедимя, что номер станет уникальным), но запмним это на будующее - убедимся в правомерности решения позже, при общении с заказчиком.

In [191]:
w_df.loc[4639,'puNumber'] = '2211543102'

In [192]:
# пробуем создать числовые индексы
w_df['puNumber'] = pd.to_numeric(w_df['puNumber'], errors='raise')

ValueError: Unable to parse string "нет" at position 1797

Видим еще одно странное значение, так же исправим произвольным номером.

In [193]:
w_df[w_df['puNumber'] == '0']

,title,puNumber,show_start_date,type,film_studio,production_country,director,producer,age_restriction,refundable_support,nonrefundable_support,budget,financing_source,ratings,genres,box_office,title_true,first_lv_desc,second_lv_desc,alter_title,title_true_len,show_start_year,show_start_month,age_limit,counts_film_studio,counts_production_country,counts_director,counts_producer


In [194]:
w_df.loc[1797,'puNumber'] = '0'

In [195]:
w_df['puNumber'] = pd.to_numeric(w_df['puNumber'], errors='raise')

In [196]:
w_df = w_df.set_index('puNumber')
w_df.sample(3).T

puNumber,121016511,221083610,112004319
title,Смертельный звонок,Суррогаты,Поэт и война
show_start_date,2011-09-09,2010-07-15,2019-08-01
type,Художественный,Художественный,Документальный
film_studio,Кор Контентс Медиа,"Мэндевиль Филмз, Тачстоун Пикчерз, Роуд Ребел,...","ООО Киновидеостудия ""Дорога времени"""
production_country,Южная Корея,США,Россия
director,Ю Сун-Донг,Джонатан Мостоу,А.Кибкало
producer,Хонг Джунг-пьё,"Тодд Либерманн, Дэвид Хоберман, Макс Хэндлман",А.Кибкало
age_restriction,«18+» - запрещено для детей,«16+» - для детей старше 16 лет,«16+» - для детей старше 16 лет
refundable_support,NaN,NaN,NaN
nonrefundable_support,NaN,NaN,NaN


### Датасет содержит много пропусков, и необходимо сделать выборку важных для нас признаков, посмотреть на объем оставшихся данных. На мой взгляд оптимальным решением будет дополнительное обогащение данных из внешних источников:

1) [Данные о рейтингах, создателях, описании фильма](https://www.kinopoisk.ru/)

2) [Данные о кассовых сборах](https://opendata.mkrf.ru/opendata/7705851331-movie_gross)

3) [Данные о гос. инвестировании](https://ekinobilet.fond-kino.ru/government-support/)

### Посчитайте динамику сборов по годам премьер фильмов. В каком году медианная сумма сборов была максимальной?

In [197]:
box_median_dict = {}

for year in sorted(w_df['show_start_year'].unique().tolist()):
    box_median = w_df[w_df['show_start_year'] == year]['box_office'].median()
    box_median_dict[year] = box_median
    print(f'{year} медианная сумма сборов: {box_median}\n')
    
print('Максимальная медиианная сумма сборов была в', max(box_median_dict, key=box_median_dict.get), ' году.')

2010 медианная сумма сборов: 1700.0

2011 медианная сумма сборов: 3000.0

2012 медианная сумма сборов: 5660.0

2013 медианная сумма сборов: 3522.5

2014 медианная сумма сборов: 18160.0

2015 медианная сумма сборов: 4920933.0

2016 медианная сумма сборов: 3846679.41

2017 медианная сумма сборов: 9968340.0

2018 медианная сумма сборов: 8891102.21

2019 медианная сумма сборов: 4627798.345

Максимальная медиианная сумма сборов была в 2017  году.


### С помощью сводной таблицы посчитайте среднюю и медианную сумму сборов для каждого года.

In [198]:
w_df['box_office_mean'] = w_df['box_office']
w_df['box_office_median'] = w_df['box_office']

pivot_mean_and_median_box = pd.pivot_table(w_df, values=['box_office_mean',
                                                         'box_office_median'], index=['show_start_year'],
                    aggfunc={'box_office_mean': np.mean,
                             'box_office_median': np.median})

pivot_mean_and_median_box.astype('int64')

,box_office_mean,box_office_median
show_start_year,,
2010,23130,1700
2011,129383,3000
2012,54339,5660
2013,161954,3522
2014,26684415,18160
2015,84940570,4920933
2016,91000569,3846679
2017,136032793,9968340
2018,104565059,8891102


### Выделите имена продюсеров из столбца producer. У кого из них самый высокий средний рейтинг фильмов?

In [199]:
producers_average_rating = w_df[['title_true','producer','ratings']]
producers_average_rating.head(2)

,title_true,producer,ratings
puNumber,,,
221048915,Открытый простор,"Дэвид Валдес, Кевин Костнер, Джейк Эбертс",7.2
111013716,Особо важное задание,NaN,6.6


In [200]:
producers_average_rating.shape

(7485, 3)

In [201]:
producers_average_rating = producers_average_rating.dropna()
producers_average_rating.shape

(6011, 3)

In [202]:
producers_average_rating['producer'] = producers_average_rating['producer'].str.split(',')
producers_average_rating.head(2)

,title_true,producer,ratings
puNumber,,,
221048915,Открытый простор,"[Дэвид Валдес, Кевин Костнер, Джейк Эбертс]",7.2
221038416,Особо опасен,"[Джим Лемли, Джейсон Нетер, Марк Е.Платт, Я...",6.8


In [203]:
producers_average_rating_explode = producers_average_rating.explode('producer').drop_duplicates()
producers_average_rating_explode = producers_average_rating_explode.reset_index()
producers_average_rating_explode.shape

(13156, 4)

In [204]:
producers_average_rating_explode.head(2)

,puNumber,title_true,producer,ratings
0,221048915,Открытый простор,Дэвид Валдес,7.2
1,221048915,Открытый простор,Кевин Костнер,7.2


In [205]:
pivot_producers_average_rating = pd.pivot_table(producers_average_rating_explode, values='ratings',
                                                                                     index=['producer'],
                    aggfunc={'ratings': np.mean})


pivot_producers_average_rating.sort_values(by='ratings', ascending=False).head(10)

,ratings
producer,
Ники Марвин,9.1
Дэвид В.Лестер,9.1
Ю.Осетинская,9.0
Э Мультимедиа Еуроп Филм,8.8
Мокер Продакшн,8.8
Куин Филмз Презент,8.8
Э Мэфилм Дайлог Студио,8.8
Карен Гилкрайст,8.8
В.Меньшов,8.8


### Результат не очень отражает суть, победители сняли лишь один фильм(

In [206]:
w_df[(w_df['producer'].notna()) & (w_df['producer'].str.contains('Ники Марвин'))].T

puNumber,121029519
title,Побег из Шоушенка
show_start_date,2019-10-18
type,Художественный
film_studio,"Уорнер Броз. Пикчерз, Кастл Рок Интертейнмент"
production_country,США
director,Фрэнк Дарабонт
producer,"Лиз Глоцер, Дэвид В.Лестер, Ники Марвин"
age_restriction,«16+» - для детей старше 16 лет
refundable_support,NaN
nonrefundable_support,NaN


# Задачи от бизнеса

1. Какой жанр имеет наибольший рейтинг? Учитывайте все жанры, к которым относится фильм. (задача на посчитать)

In [208]:
w_df_ganres = w_df[~w_df['genres'].isna()]

In [211]:
w_df_ganres['genres_list'] = w_df_ganres['genres'].str.split(',')

<ipython-input-211-3361362ab258>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w_df_ganres['genres_list'] = w_df_ganres['genres'].str.split(',')


In [212]:
w_df_ganres_explode = w_df_ganres.explode('genres_list').drop_duplicates()
w_df_ganres_explode = w_df_ganres_explode.reset_index()
w_df_ganres_explode.shape

(15211, 31)

In [214]:
top_ganres = pd.pivot_table(w_df_ganres_explode, values='ratings',
                                                                                     index=['genres_list'],
                    aggfunc={'ratings': np.mean})


top_ganres.sort_values(by='ratings', ascending=False).head(10)

,ratings
genres_list,
новости,7.800000
аниме,7.443836
документальный,7.202632
концерт,7.165000
короткометражка,7.066038
биография,7.040968
мюзикл,6.993636
спорт,6.961458
вестерн,6.906250


In [220]:
w_df[(w_df['genres'].notna()) & (w_df['genres'].str.contains('новости'))].T

puNumber,126003515
title,Реальная цена моды
show_start_date,2015-11-25
type,Прочие
film_studio,"Антолд Креатив, Лайф Ис Май Муви Интертейнмент..."
production_country,Великобритания - Франция - Италия - Индия - Да...
director,Эндрю Морган
producer,"Винсент Витторио, Ливия Джуджолли, Майкл Росс,..."
age_restriction,«16+» - для детей старше 16 лет
refundable_support,NaN
nonrefundable_support,NaN


### <font color='green'>выделил задачки с которыми хочу поработать</font>

2. Велик ли объем господдержки в фильмах? (надо посчитать долю для каждого фильма и обосновать много это или мало)

3. Какие фильмы лучше - с господдержкой или без? (надо предложить и обосновать понятие "лучше")

4. Составьте список успешных режиссеров. (предложить и обосновать понятие успешности)

### <font color='green'>5. Каких продюсеров можно назвать "восходящими звездами кинематографа?" (предложить показатель успешности и проанализовать динамику этого показателя)</font>

### <font color='green'>6. Влияет ли возрастное ограничение на успех фильма?</font>

### <font color='green'>7. Решите проблемы с пропусками в данных с помощью внешних источников.</font>

### <font color='green'>8. Сейчас наши коллеги-аналитики работают с данными в Экселе и тратят много времени на обработку. Данные проекта - результат из работы. Ваша задача указать проблемы в данных, которые они упустили. И показать преимущества pandas для подобных задач.</font>

### <font color='green'>9. Постройте модель предсказания будущего рейтинга фильма или успешности в финансовых показателях.</font>


## Пока не до конца успел сформулировать свои дальнейшие шаги, в общих чертах:

1. хочу разместить БД по фильмам в Google Cloude;

2. в БД накапливать/парсить данные - использовать Silenium;

3. там же собрать модель для оценки успешных продюссеров и продюссерских коллективов.